In [1]:
!pip install llmlingua openai

In [2]:
pip show llmlingua

Name: llmlingua
Version: 0.2.2
Summary: To speed up LLMs' inference and enhance LLM's perceive of key information, compress the prompt and KV-Cache, which achieves up to 20x compression with minimal performance loss.
Home-page: https://github.com/microsoft/LLMLingua
Author: The LLMLingua team
Author-email: hjiang@microsoft.com
License: MIT License
Location: /usr/local/lib/python3.10/dist-packages
Requires: accelerate, nltk, numpy, tiktoken, torch, transformers
Required-by: 


In [3]:
import llmlingua
help(llmlingua.PromptCompressor.compress_prompt)

Help on function compress_prompt in module llmlingua.prompt_compressor:

compress_prompt(self, context: List[str], instruction: str = '', question: str = '', rate: float = 0.5, target_token: float = -1, iterative_size: int = 200, force_context_ids: List[int] = None, force_context_number: int = None, use_sentence_level_filter: bool = False, use_context_level_filter: bool = True, use_token_level_filter: bool = True, keep_split: bool = False, keep_first_sentence: int = 0, keep_last_sentence: int = 0, keep_sentence_number: int = 0, high_priority_bonus: int = 100, context_budget: str = '+100', token_budget_ratio: float = 1.4, condition_in_question: str = 'none', reorder_context: str = 'original', dynamic_context_compression_ratio: float = 0.0, condition_compare: bool = False, add_instruction: bool = False, rank_method: str = 'llmlingua', concate_question: bool = True, context_segs: List[str] = None, context_segs_rate: List[float] = None, context_segs_compress: List[bool] = None, target_cont

In [4]:
import pandas as pd

In [5]:
df = pd.read_csv('/content/Tweets Sentiment Analysis.csv')

In [6]:
import getpass
import openai

secret_key = getpass.getpass('Digite sua chave da OpenAI: ')
openai.api_key = secret_key

Digite sua chave da OpenAI: ··········


In [7]:
client = openai.Client(api_key=secret_key)

In [8]:
def count_tokens(text):
    return len(text.split())

In [9]:
def analyze_sentiment_openai(text):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a sentiment analysis expert. Please classify the tweet as 'positive', 'negative', or 'neutral'. Do not provide any other form of response."},
            {"role": "user", "content": f"Analyze the sentiment of the following text: '{text}'"}
        ],
        max_tokens=300,
        temperature=0.3,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )
    sentiment = response.choices[0].message.content.strip()
    tokens = count_tokens(f"Analyze the sentiment of the following text: '{text}'")
    return sentiment, tokens

In [10]:
from llmlingua import PromptCompressor

In [11]:
llm_lingua = PromptCompressor()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuratio

In [12]:
def analyze_sentiment_llmlingua(text):
    context = [f"Analyze the sentiment of the following text: '{text}'"]
    instruction = "You are a sentiment analysis expert. Please classify the text as 'positive', 'negative', or 'neutral'. Do not provide any other form of response."

    compressed_prompt = llm_lingua.compress_prompt(
        context=context,
        instruction=instruction,
        question="",
        rate=0.5,  # Taxa de compressão desejada (50%)
        target_token=100  # Reduzindo o número de tokens alvo
    )

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a sentiment analysis expert. Please classify the tweet as 'positive', 'negative', or 'neutral'. Do not provide any other form of response."},
            {"role": "user", "content": compressed_prompt['compressed_prompt']}
        ],
        max_tokens=300,
        temperature=0.3,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )
    sentiment = response.choices[0].message.content.strip()
    tokens = count_tokens(response.choices[0].message.content.strip())
    return sentiment, tokens, compressed_prompt

In [13]:
texts = df['text'].tolist()
original_sentiments = df['sentiment'].tolist()

# Analisando com OpenAI
results_openai = [analyze_sentiment_openai(text) for text in texts]

# Analisando com LLMLingua
results_llmlingua = [analyze_sentiment_llmlingua(text) for text in texts]

# Criando um DataFrame para comparação
df_results = pd.DataFrame({
    'Text': texts,
    'Original_sentiment': original_sentiments,
    'Sentiment_OpenAI': [result[0] for result in results_openai],
    'Tokens_OpenAI': [result[1] for result in results_openai],
    'Sentiment_LLMLingua': [result[0] for result in results_llmlingua],
    'Tokens_LLMLingua': [result[1] for result in results_llmlingua]
})

df_results.to_csv('results.csv', index=False)

In [14]:
df_results.head()

,Text,Original_sentiment,Sentiment_OpenAI,Tokens_OpenAI,Sentiment_LLMLingua,Tokens_LLMLingua
0,i checked. we didn`t win,neutral,negative,12,negative,1
1,"sitting here, waiting to go to the gym.....alm...",neutral,Negative,29,Negative,1
2,i feel sorry for my bestie good luck ali!,neutral,positive,16,positive,1
3,airsoft is so much fun! i play with my brother...,positive,Positive,23,Positive,1
4,alice doesn`t know what to wear to the cinema <3,negative,Neutral,17,Neutral,1
